In [135]:
# install dependencies
%pip install openai jinja2 ipywidgets d20 python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [136]:
# load dotenv file
import os

from dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY") is None:
    raise Exception("missing OPENAI API KEY...")

In [137]:
# load config file
import tomllib

config_name = "config.toml"
config:dict[str, any] = None
with open(config_name, "rb") as f:
    config = tomllib.load(f)

print(f"{config_name} loaded: v{config["version"]}")

config.toml loaded: v0.0.2


In [138]:
# load config
from jinja2 import BaseLoader, Environment

meta = config.get("meta", None)
if meta.get("memories") is None:
    meta["memories"] = []

current_round = 0
messages:dict[str, str] = []

def next_round():
    global current_round, messages
    messages = [] # reset messages
    current_round += 1
    for msg in config["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")
        
        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)

        messages.append({"role": role, "content": content})
        # print("------")
    
    for msg in config["rounds"][current_round - 1]["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")

        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)
       
        messages.append({"role": role, "content": content})
        # print("------")

next_round()

In [139]:
# start interactive chat
import re
import functools

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import d20

def count_user_actions():
    count = 0
    for msg in messages:
        if msg["role"] == "user":
            count += 1
    return count

def is_final_round():
    return  ", FINAL ROUND" if  current_round == len(config["rounds"]) else ""

def on_custom_input(b, input:widgets.Text):
    role = "user"
    content = f"{input.value} (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_action(b, index:str):
    role = "user"
    content = f"I have chosen action {index}. (TURN: {count_user_actions()+1}{is_final_round()})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def on_skill(b, skill:str, difficulty:str):
    role = "user"
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. (TURN: {count_user_actions()+1}, {is_final_round()})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_next_round(b):
    next_round()
    print("ROUND:", current_round)
    print(f"{messages[-1]["role"].upper()}:")
    print(messages[-1]["content"])
    parse_message(messages[-1]["content"])

# parse message into key-value pairs
pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
action_pattern = re.compile(r"(\d+). *(.+)")
def parse_message(content):
    parsed: dict = {}
    current_key = None
    lines = content.splitlines()
    
    # parse the content line by line
    for line in lines:
        res = pattern.search(line)
        if res is not None:
            current_key = res.group(1)
            if res.group(2) is not None:
                parsed[current_key] = res.group(2)
            else:
                parsed[current_key] = ""
        elif current_key is not None:
            parsed[current_key] += "\n" + line
    
    keys = parsed.keys()
    # possible actions
    vbox = widgets.HBox()
    if "possible actions" in keys:
        ma = action_pattern.finditer(parsed["possible actions"])
        for m in ma:
            action_index=m.group(1)
            action_button = widgets.Button(description=action_index)
            action_button.on_click(functools.partial(on_action, index=action_index))
            vbox.children += (action_button,)
        custom_action_button = widgets.Button(description="CUSTOM")
        custom_action_button.on_click(functools.partial(on_custom_action))
        
        vbox.children += (custom_action_button,)
        display(vbox)
    
    # skill check
    if "skill" in keys and "difficulty" in keys:
        skill = parsed.get("skill")
        difficulty = parsed.get("difficulty")
        skill_button = widgets.Button(description=f"SKILL CHECK - {skill.upper()}", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        display(skill_button)
    
    if "round summary" in keys:
        summary = parsed["round summary"]
        next_round_button = widgets.Button(description="NEXT ROUND")
        meta["memories"].append(summary)
        
        next_round_button.on_click(on_next_round)
        display(next_round_button)
    
    if "ending summary" in keys:
        print("\n\nGAME OVER\n\n(try to re-run the cell to start a new game)")


def chat():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )
    print("\nASSISTANT:")
    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            print(delta, end="")
            generated += delta
    messages.append({"role": "assistant", "content": generated})
    parse_message(messages[-1]["content"])

print("ROUND:", current_round)
print(f"{messages[-1]["role"].upper()}:")
print(messages[-1]["content"])
# parse the last message
parse_message(messages[-1]["content"])

ROUND: 1
ASSISTANT:
You are standing in front of the haunted mansion. 
The mansion is old and decrepit, with ivy growing up the walls and the windows boarded up. 
The front door is slightly ajar, inviting you inside. 

What do you do?

possible actions:
1. Enter the mansion.
2. Take some photos of the mansion's exterior.
3. Talk to the butler. (may need a persuasion skill check)


USER ACTION: I have chosen action 1. (TURN: 1)

ASSISTANT:
You push the front door open, which creaks loudly, echoing through the seemingly empty mansion. The air inside is musty, and dust particles dance in the beams of light filtering in through the cracks in the boarded-up windows. The foyer is grand, with a large staircase leading to the upper floors and several doors leading to various rooms on the ground floor. 

As you step inside, you notice that one of the doors is slightly ajar, leading to what appears to be the dining room. 

possible actions:
1. Explore the dining room.
2. Head up the staircase to the upper floors.
3. Investigate the other rooms on the ground floor.

USER ACTION: I have chosen action 1. (TURN: 2)

ASSISTANT:
As you cautiously approach and enter the dining room, you're greeted by a grand but neglected space. A long dining table, covered in a thick layer of dust, dominates the room, surrounded by chairs that have seen better days. Faint, eerie light filters through the dirty windows, casting shadows across the room.

Suddenly, a large, shadowy figure of a dog darts across the room, disappearing through a door on the other side. The sight is startling, but before fear can take hold, you remind yourself of the task at hand.

skill: sanity
difficulty: easy

Button(description='SKILL CHECK - SANITY', layout=Layout(width='auto'), style=ButtonStyle())

USER ACTION: I am making a skill check using SANITY against a difficulty of EASY.
And I rolled a 53 for a result of SUCCESS. (TURN: 3, )

ASSISTANT:
Your heart races as you witness the spectral canine, but your resolve steadies quickly. Drawing upon your experience and rational mind, you convince yourself that there must be a logical explanation for what you've just seen. Perhaps it was a trick of the light or an animal that has made its home in the abandoned mansion. With your sanity intact, you decide it's best to continue your investigation.

After the unsettling encounter, you take a moment to observe the room more closely. You notice several portraits on the walls, depicting individuals you assume were once residents of the mansion. There's also a sideboard that looks like it might contain silverware or other items of interest.

possible actions:
1. Examine the portraits more closely.
2. Investigate the sideboard for any useful items.
3. Leave the dining room and explore the room 

...

USER ACTION: I have chosen action 2. (TURN: 1, FINAL ROUND)

ASSISTANT:
You carefully set up your camera, adjusting the settings to capture the dimly lit entrance of the basement. The flash momentarily illuminates the darkened area, revealing the old, worn steps and the thick cobwebs that adorn the doorway. It's a chilling sight, but you manage to capture a few photographs that might prove useful later, either for your investigation or as evidence of your journey into the unknown.

possible actions:
1. Enter the basement carefully, watching your step.
2. Examine the photos you just took for any clues or details.
3. Call out to see if anyone or anything responds.

USER ACTION: I have chosen action 1. (TURN: 2, FINAL ROUND)

ASSISTANT:
You take a deep breath and step into the basement, the air cool and musty around you. The stairs creak under your weight as you descend, your hand grazing the wall for balance. The dim light from the entrance fades, leaving you in near darkness, but you can just make out shapes in the gloom. Ahead, you see the silhouette of a statue, likely the one mentioned in your briefing. It stands ominously in the center of the room, its details obscured by the shadows.

possible actions:
1. Approach the statue to investigate further.
2. Use your flashlight to illuminate the basement and get a better view of your surroundings.
3. Call out, announcing your presence to see if it triggers any response.

USER ACTION: I have chosen action 2. (TURN: 3, FINAL ROUND)

ASSISTANT:
You pull out your flashlight and flick it on, the beam cutting through the darkness of the basement. The light reveals a cluttered room filled with old furniture covered in dust sheets, boxes stacked against the walls, and various oddities that seem to have been collected over many years. The statue stands in the center of the room, now fully illuminated. It depicts a creature that is part human, part something else, with tentacles where its mouth should be. The craftsmanship is disturbingly detailed, giving the statue an unnervingly lifelike appearance.

possible actions:
1. Approach the statue to investigate further.
2. Search the room for any documents or objects that might relate to the statue or the curse.
3. Take photos of the statue and the room for your records.

USER ACTION: I have chosen action 2. (TURN: 4, FINAL ROUND)

ASSISTANT:
As you explore the basement, you carefully sift through the clutter, looking for anything that might shed light on the statue or the mansion's curse. Your search uncovers a dusty, leather-bound journal hidden beneath a pile of old newspapers. The cover is worn, and the pages are yellowed with age, but it's intact. You also find a small, intricately carved box that seems out of place among the mundane items scattered around the basement.

possible actions:
1. Open the journal and read its contents.
2. Examine the carved box more closely.
3. Take photos of the journal and box before interacting with them.

USER ACTION: I have chosen action 2. (TURN: 5, FINAL ROUND)

ASSISTANT:
You carefully pick up the intricately carved box, noticing the fine details and symbols etched into its surface. The craftsmanship suggests it's of significant importance. With a gentle touch, you manage to open it, revealing a set of old, faded photographs and a small, metallic key with peculiar engravings that match some of the symbols on the statue. The photographs depict the mansion in earlier days, showing a family that you presume were the original owners. Among them, an old woman stands out, her gaze piercing even through the aged photo.

As you ponder the significance of these items, you hear footsteps approaching. Turning around, you see an old woman, resembling the one from the photographs, standing at the bottom of the stairs. She smiles warmly, yet there's a hint of sadness in her eyes.

"I see you've found my family's belongings," she begins, her voice echoing slightly in the spacious basement. "You've